In [13]:
import pandas as pd
import numpy as np
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)


In [14]:
# The human freedom index dataset for countries.
hfi = pd.read_csv("../data/human-freedom.csv")

# The quality of life dataset for a subset of cities.
qol = pd.read_csv("../data/quality-of-life.csv")

# Climate data for cities around the world.
climate = pd.read_json("../data/wiki-climate.json")

for df in [hfi, qol, climate]:
    df.columns = df.columns.str.lower().str.replace(" ", "_").str.replace("&", "and")


### Quality of life dataset

The QoL dataset contains information about specific cities (which is of interest to our application). However, the range of countries described by these cities is smaller than that of the Human Freedom Index dataset. Only the countries within the QoL dataset should be used within the final processed dataset.


In [15]:
qol = qol.rename(columns={"ua_name": "city", "ua_country": "country"})
qol = qol[
    [
        "city",
        "country",
        "housing",
        "cost_of_living",
        "startups",
        "venture_capital",
        "travel_connectivity",
        "commute",
        "business_freedom",
        "safety",
        "healthcare",
        "education",
        "environmental_quality",
        "economy",
        "taxation",
        "internet_access",
        "leisure_and_culture",
        "tolerance",
        "outdoors",
    ]
]
qol["city"] = qol["city"].str.strip()
qol["country"] = qol["country"].str.strip()
qol = qol.round(3)
qol.head()


,city,country,housing,cost_of_living,startups,venture_capital,travel_connectivity,commute,business_freedom,safety,healthcare,education,environmental_quality,economy,taxation,internet_access,leisure_and_culture,tolerance,outdoors
0,Aarhus,Denmark,6.132,4.015,2.827,2.512,3.536,6.312,9.940,9.617,8.704,5.366,7.633,4.887,5.068,8.373,3.187,9.739,4.130
1,Adelaide,Australia,6.310,4.692,3.136,2.640,1.777,5.336,9.400,7.926,7.937,5.142,8.331,6.070,4.588,4.341,4.328,7.822,5.531
2,Albuquerque,United States,7.262,6.059,3.772,1.493,1.456,5.056,8.671,1.344,6.430,4.152,7.320,6.514,4.346,5.396,4.890,7.028,3.516
3,Almaty,Kazakhstan,9.282,9.333,2.458,0.000,4.592,5.871,5.568,7.309,4.546,2.283,3.857,5.269,8.522,2.886,2.937,6.540,5.500
4,Amsterdam,Netherlands,3.053,3.824,7.972,6.107,8.324,6.118,8.837,8.504,7.907,6.180,7.597,5.053,4.955,4.523,8.874,8.368,5.307


### Human freedom index dataset

This dataset contains freedom indices for countries, including human freedom (HF), personal freedom (PF) and economic freedom (EF).


In [16]:
hfi = hfi.rename(columns={"countries": "country"})
hfi = hfi[hfi["year"] == 2020]
hfi = hfi[["country", "hf_score", "ef_score", "pf_score"]]
hfi = hfi.rename(
    columns={
        "hf_score": "human_freedom",
        "ef_score": "economic_freedom",
        "pf_score": "personal_freedom",
    }
)
hfi["country"] = hfi["country"].str.strip()
hfi = hfi[hfi["country"].isin(qol["country"])]
hfi.head()


,country,human_freedom,economic_freedom,personal_freedom
3,Argentina,6.99,4.87,8.51
4,Armenia,8.14,7.84,8.35
5,Australia,8.51,8.04,8.85
6,Austria,8.26,7.56,8.76
7,Azerbaijan,5.70,6.21,5.35


### Wikipedia Climate dataset

Climate data is very noisy and sparse. It is consistent only for larger cities. To sanitise, we remove extra characters and accents from city names and extract only the desired columns.


In [17]:
climate = climate.rename(columns={"name": "city"})
climate["city"] = (
    climate["city"]
    .str.normalize("NFKD")
    .str.encode("ascii", errors="ignore")
    .str.decode("utf-8")
)
climate["city"] = climate["city"].apply(lambda x: x.split(",")[0].split("(")[0])
climate["city"] = climate["city"].str.strip()
climate["country"] = climate["country"].str.strip()
climate = climate.rename(
    columns={
        "year_high_c": "max_temp",
        "year_precipitation_mm": "precipitation",
        "year_low_c": "min_temp",
        "year_mean_c": "mean_temp",
        "gps_lon": "lon",
        "gps_lat": "lat",
        "city_wd": "wikidata",
    }
)
climate = climate[
    [
        "city",
        "country",
        "population",
        "wikidata",
        "lon",
        "lat",
        "max_temp",
        "mean_temp",
        "min_temp",
        "precipitation",
    ]
]
climate[["mean_temp", "min_temp", "max_temp"]] = (
    climate[["mean_temp", "min_temp", "max_temp"]].replace("", np.nan).astype("float")
)
climate["mean_temp"] = climate["mean_temp"].fillna(
    (climate["max_temp"] + climate["min_temp"]) / 2
)
climate["temp_range"] = climate["max_temp"] - climate["min_temp"]
climate = climate.drop(columns=["max_temp", "min_temp"])
climate = climate.sort_values(by="population", ascending=False)
climate = climate.drop_duplicates(subset=["city", "country"], keep="first")
climate.head()


,city,country,population,wikidata,lon,lat,mean_temp,precipitation,temp_range
0,Delhi,India,26495000,http://www.wikidata.org/entity/Q1353,77.216667,28.666667,25.1,790,12.3
1,Shanghai,China,23390000,http://www.wikidata.org/entity/Q8686,121.466667,31.166667,17.1,1166.1,6.5
2,Beijing,China,21710000,http://www.wikidata.org/entity/Q956,116.391389,39.905000,12.5,571.8,10.6
3,Lagos,Nigeria,21324000,http://www.wikidata.org/entity/Q8673,3.400000,6.450000,26.8,1506.6,8.0
4,Karachi,Pakistan,14910352,http://www.wikidata.org/entity/Q8660,67.010000,24.860000,26.0,NaN,11.4


### Creating a combined dataset

We combine all of the sanitised datasets using QoL city name as a primary key for the climate data, and QoL country as a primary key for the human freedom data.


In [18]:
# Merge the cleaned datasets
combined = pd.merge(qol, hfi, on="country", how="left")
combined = pd.merge(combined, climate, on=["city", "country"], how="left")
combined = combined.rename(columns={"city": "name"})

# Rearrange the columns and drop null values
cols = combined.columns.tolist()
cols = cols[:2] + cols[-7:] + cols[2:-7]
combined = combined[cols]

# Convert column names to camel case so that they are JSON friendly
combined.columns = (
    combined.columns.str.replace("_", " ").str.title().str.replace(" ", "")
)
combined.columns = combined.columns.str[0].str.lower() + combined.columns.str[1:]

# Drop null rows (usually it's precipitation missing)
combined = combined.dropna()
combined["population"] = combined["population"].astype(int)

# Compute ranked data for each column
rankedData = combined.iloc[:, 6:]
rankedData["population"] = combined["population"]
rankedData = rankedData.rank(axis=0, method="dense", ascending=False).astype(int)
rankedData.columns = rankedData.columns + "Rank"
combined = pd.concat([combined, rankedData], axis=1)
combined["id"] = combined.index

# Save the data as a CSV dataset for further use.
combined.to_csv("../data/preprocessed.csv", index=False)
combined.to_json("../server/db.json", orient="records", indent=2)

combined.head()


,name,country,population,wikidata,lon,lat,meanTemp,precipitation,tempRange,housing,...,taxationRank,internetAccessRank,leisureAndCultureRank,toleranceRank,outdoorsRank,humanFreedomRank,economicFreedomRank,personalFreedomRank,populationRank,id
0,Aarhus,Denmark,269022,http://www.wikidata.org/entity/Q25319,10.209722,56.156389,7.7,725,7.6,6.132,...,41,13,175,1,122,3,2,8,158,0
2,Albuquerque,United States,559277,http://www.wikidata.org/entity/Q34804,-106.616667,35.116667,13.9,240.1,12.9,7.262,...,53,104,134,115,145,18,3,23,108,2
3,Almaty,Kazakhstan,1703500,http://www.wikidata.org/entity/Q35493,76.900000,43.250000,10.0,684,10.8,9.282,...,5,177,178,134,40,55,28,55,40,3
4,Amsterdam,Netherlands,851573,http://www.wikidata.org/entity/Q727,4.900000,52.383333,10.2,838.2,7.4,3.053,...,43,142,9,40,56,8,11,9,70,4
7,Ankara,Turkey,5270575,http://www.wikidata.org/entity/Q3640,32.866667,39.866667,12.0,402.2,11.5,9.928,...,55,184,10,177,64,64,49,59,19,7
